In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-epi2okeh/unsloth_da0c7685dfb84d78b4e487365f384abe
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-epi2okeh/unsloth_da0c7685dfb84d78b4e487365f384abe
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 12.5 MB/s eta 0:00:00
   ━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 23.4 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.13.0
    Uninstalling trl-0.13.0:
      Successfully uninstalled trl-0.13.0


**CONSTANTES**

In [3]:
DATA_PATH = "/content/drive/MyDrive/TECH_FASE3/trn_parcial.json"
MODEL_PATH = "/content/drive/MyDrive/TECH_FASE3/lora_model"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/TECH_FASE3/trn_formatado.json"
KEY = "c711172f16b8fa71cadb5871200e8bbf5094e720"

**Quantização**

Quantização é uma técnica para reduzir a precisão dos números usados para
representar os pesos de um modelo de rede neural. Isso é feito para reduzir os
requisitos de memória e acelerar o processamento, permitindo que os modelos sejam
executados mais eficientemente em hardware com recursos limitados, como
dispositivos móveis. Em nosso código, utilizamos a quantização de 4 bits, indicando
que os pesos do modelo são representados com apenas 4 bits, o que é
significativamente menor que a precisão típica de 32 bits usada em modelos de ponto
flutuante. Dessa forma, conseguimos treinar o modelo sem necessitar de um hardware
avançado, o que pode ser demasiadamente caro e inacessível.

In [22]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

**LLaMA**

O LLaMA, da Meta AI, é um modelo de linguagem projetado para eficiência e
adaptabilidade, capaz de executar tarefas complexas de processamento de linguagem
natural, como tradução, resposta a perguntas e sumarização. Com diversas versões
disponíveis para diferentes necessidades computacionais, o LLaMA oferece uma
solução acessível para a implantação de tecnologias de linguagem avançadas em
uma variedade de plataformas.
Por ser open source e um dos mais utilizados para tarefas que envolvem o
processamento de linguagem natural, optamos por usar este modelo em nossa aula.

In [23]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


**PEFT (Parameter Efficient Fine-Tuning)**

É uma abordagem para ajustar modelos pré-treinados de uma maneira que seja
computacionalmente eficiente. Isso geralmente envolve alterar apenas uma pequena
fração dos pesos do modelo durante o fine-tuning, em vez de ajustar todo o modelo. Essa técnica reduz a carga computacional e o uso de memória durante o treinamento, tornando o fine-tuning mais acessível e prático em uma variedade de situações.

**LoRA (Low-Rank Adaptation)**

LoRA é uma técnica específica dentro do espectro de métodos PEFT. Ela
modifica apenas as camadas lineares do modelo, inserindo transformações de baixo
posto que são treinadas enquanto os outros pesos são mantidos congelados. Isso
permite que o modelo seja adaptado para tarefas específicas com apenas um
pequeno acréscimo nos parâmetros, mantendo a maioria do conhecimento prétreinado
intacto e reduzindo a quantidade de dados necessários para efetuar o ajuste.

In [24]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [25]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context about a store product. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

**3. Chamada do Foundation Model**

Importe o foundation model que será utilizado e faça um teste
apresentando o resultado atual do modelo antes do treinamento (para que
se obtenha uma base de análise após o fine-tuning), e então será possível
avaliar a diferença do resultado gerado.

In [26]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "DESCRIBE THIS TITLE",
        "Collins Spanish Dictionary.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context about a store product. Write a response that appropriately completes the request.\n\n### Instruction:\nDESCRIBE THIS TITLE\n\n### Input:\nCollins Spanish Dictionary.\n\n### Response:\nThis dictionary is useful for anyone who wants to learn Spanish. It has a variety of words and phrases that will help you communicate with people in Spain or Latin America.\n\n### Correct Response:\nThis dictionary is useful for anyone who wants to learn Spanish. It has a variety of words and phrases that will help you communicate with people']

**TextStreamer**

TextStreamer para inferência contínua - assim você pode ver a geração token por token, em vez de esperar o tempo todo!

In [27]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "DESCRIBE THE TITLE",
        "Collins Spanish Dictionary.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context about a store product. Write a response that appropriately completes the request.

### Instruction:
DESCRIBE THE TITLE

### Input:
Collins Spanish Dictionary.

### Response:
Collins Spanish Dictionary. It is a dictionary that is used to look up the definition of words in Spanish. It is a popular dictionary that is used by many people who are learning Spanish.<|end_of_text|>


**Preparação do Dataset:**

Faça o download do dataset AmazonTitles-1.3MM e utilize o arquivo
“trn.json”. Nele, você utilizará as colunas “title” e “content”, que contêm título e
descrição respectivamente. Prepare os prompts para o fine-tuning garantindo
que estejam organizados de maneira adequada para o treinamento do modelo
escolhido. Limpe e pré-processe os dados conforme necessário para o modelo
escolhido.

In [28]:
def format_dataset_into_model_input(data):

    def separate_text(full_text):
        instruction = "DESCRIBE THIS TITLE"
        input_text = full_text["title"].strip()
        response = full_text["content"].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for prompt in data['train']['input']:
        instruction, input_text, response = separate_text(prompt)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [29]:
import unicodedata

def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def isNotBlank (myString):
    return bool(myString and myString.strip())

def load_data():
  with open(DATA_PATH, 'r') as file:
    data = file.readlines()

  # Inicializando as listas para armazenar os dados
  instructions = []
  inputs = []
  outputs = []

  for line in data:
    item = json.loads(line)
    input = item.get('title')
    output = item.get('content')
    if isNotBlank(input) and isNotBlank(output):
      instructions.append("DESCRIBE THIS TITLE")
      inputs.append(strip_accents(input.strip()))
      outputs.append(strip_accents(output.strip()))

  # Criando o dicionário final
  formatted_data = {
      "instruction": instructions,
      "input": inputs,
      "output": outputs
  }

  # Salvando o resultado em um arquivo JSON
  with open(OUTPUT_PATH_DATASET, 'w') as output_file:
      json.dump(formatted_data, output_file, indent=4)

  print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")


In [30]:
load_data()

Dataset salvo em /content/drive/MyDrive/TECH_FASE3/trn_formatado.json


In [31]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [32]:
from datasets import load_dataset

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 75062
})

In [33]:
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/75062 [00:00<?, ? examples/s]

In [43]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 75062
})

In [34]:
import torch

max_seq_length = 1024

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/75062 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [35]:
# Free up memory before training
torch.cuda.empty_cache()

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 75,062 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.763400
2,4.176200
3,3.491900
4,3.716300
5,3.334500
6,3.171600
7,3.407200
8,2.924400
9,3.117700
10,2.701200


In [36]:
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

('/content/drive/MyDrive/TECH_FASE3/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/TECH_FASE3/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/TECH_FASE3/lora_model/tokenizer.json')

In [37]:
torch.cuda.empty_cache()

if True:
    from unsloth import FastLanguageModel
    from transformers import BitsAndBytesConfig # Import BitsAndBytesConfig

    # Create a BitsAndBytesConfig object with CPU offloading enabled
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=load_in_4bit,
        #llm_int8_enable_fp32_cpu_offload=True,
    )

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = MODEL_PATH,
        max_seq_length = max_seq_length,
        dtype = dtype,
        # Pass the quantization_config to from_pretrained
        quantization_config=quantization_config,
        # Pass a device_map to load the model in 4-bit with offloading
        device_map="auto", # or a custom device_map if needed
        # Enable CPU offloading for the modules that cannot fit on the GPU
        trust_remote_code=True, # this might be needed depending on the model
        load_in_8bit=False,
    )
    FastLanguageModel.for_inference(model)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [38]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "DESCRIBE THIS TITLE",
        "Collins Spanish Dictionary.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context about a store product. Write a response that appropriately completes the request.\n\n### Instruction:\nDESCRIBE THIS TITLE\n\n### Input:\nCollins Spanish Dictionary.\n\n### Response:\nCollins Spanish Dictionary is a bilingual Spanish-English dictionary with more than 100,000 words and phrases and 150,000 translations, as well as a wide range of additional features to help you find the exact translation you need. The latest edition has been completely revised and updated to include the latest words and expressions,']

In [39]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "DESCRIBE THIS TITLE",
        "Collins Spanish Dictionary.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context about a store product. Write a response that appropriately completes the request.

### Instruction:
DESCRIBE THIS TITLE

### Input:
Collins Spanish Dictionary.

### Response:
Collins Spanish Dictionary is the most comprehensive and up-to-date Spanish dictionary available. It is designed for use by all levels of Spanish speakers, from intermediate to advanced. The dictionary is easy to use and ideal for both reference and study purposes. It includes 120,000 references and 140,000 translations, and a new section on Spanish life and culture. This edition also includes a 16-page color section on the Spanish-speaking world.<|end_of_text|>
